In [1]:
# problem instance settings
state = 'LA'
deviation = 0.10  # 0.10 = 10.00%
minority = 'Black'
level = 'tract'
district_type = 'SS'

# short bursts settings
burst_length = 10 
total_steps = 10000 
num_bursts = round(total_steps / burst_length) 

In [2]:
# For reproducibility, use random seed. Choose same as:
#  https://github.com/vedikavish1/Georgia-Redistricting/blob/main/Markov_Chain_bg.py
import random 
random.seed(48)

In [3]:
# read tract graph
filepath = 'C://districting-data-2020//'
filename = state + '_' + level + '.json'

from gerrychain import Graph
G = Graph.from_json( filepath + filename )

In [4]:
from util import get_census_codes
codes = get_census_codes(minority)

for i in G.nodes:
    G.nodes[i]['TOTPOP'] = G.nodes[i]['P0010001']
    G.nodes[i]['VAP'] = G.nodes[i]['P0030001']
    G.nodes[i]['MVAP'] = sum( G.nodes[i][code] for code in codes )
    
from util import number_of_districts
k = number_of_districts[state,district_type]
ideal_population = sum( G.nodes[i]['TOTPOP'] for i in G.nodes ) / k

In [5]:
# Import attributes of gerrychain that we need to create the initial partition: 
from gerrychain import Graph, Partition, updaters, constraints 
from gerrychain.updaters import cut_edges, Tally
from gerrychain.tree import recursive_tree_part

initial_plan = Partition(G, 
assignment = recursive_tree_part(G, range(k), ideal_population, "TOTPOP", 0.9*deviation/2, 10), 
updaters={
    "cut_edges": cut_edges, 
    "population": Tally("TOTPOP",alias="population"), 
    "MVAP": Tally("MVAP"), 
    "VAP": Tally("VAP"), 
})

In [6]:
def print_stats(plan):
    num_majority_minority = 0
    print('# mvap vap mvap%')
    for district in plan.parts:
        mvap = plan['MVAP'][district]
        vap = plan['VAP'][district]
        print(district, mvap, vap, round(100*mvap/vap,2),'%')
        if mvap >= 0.5 * vap:
            num_majority_minority += 1
    print("Number of majority-minority districts:",num_majority_minority)

In [7]:
print_stats(initial_plan)

# mvap vap mvap%
0 20064 92905 21.6 %
1 21002 89416 23.49 %
2 40207 92316 43.55 %
3 35105 89987 39.01 %
4 12698 96502 13.16 %
5 32481 93771 34.64 %
6 31651 97596 32.43 %
7 19074 91047 20.95 %
8 40335 88521 45.57 %
9 32283 90769 35.57 %
10 26831 88760 30.23 %
11 27416 87562 31.31 %
12 13054 101173 12.9 %
13 43586 90410 48.21 %
14 27867 95770 29.1 %
15 31096 88751 35.04 %
16 20046 91822 21.83 %
17 32588 91469 35.63 %
18 13652 86988 15.69 %
19 28728 89640 32.05 %
20 41182 102507 40.17 %
21 34154 91059 37.51 %
22 15366 89764 17.12 %
23 23965 89918 26.65 %
24 24569 89498 27.45 %
25 30911 93388 33.1 %
26 34741 92990 37.36 %
27 19734 87504 22.55 %
28 65216 90935 71.72 %
29 11224 88171 12.73 %
30 12348 86877 14.21 %
31 24085 92035 26.17 %
32 29780 93409 31.88 %
33 63716 93832 67.9 %
34 11582 89069 13.0 %
35 30453 90731 33.56 %
36 25339 90917 27.87 %
37 38351 92085 41.65 %
38 29289 90684 32.3 %
Number of majority-minority districts: 2


In [8]:
## Import additional attributes from Gerrychain required to run the short bursts: 
from gerrychain.proposals import recom # propose_random_flip, 
from functools import partial
from gerrychain.accept import always_accept

initial_state = initial_plan   # initial_state specifies the seed plan the short bursts start from

compactness_bound = constraints.UpperBound(
    lambda p: len(p["cut_edges"]), 
    2*len(initial_plan["cut_edges"])
)

pop_constraint = constraints.within_percent_of_ideal_population(initial_plan, deviation/2)

proposal1 = partial(recom, pop_col="TOTPOP", pop_target=ideal_population, epsilon=deviation/2, node_repeats=1)

In [9]:
from gerrychain import MarkovChain

incumbent_plan = initial_plan
print("Burst \t Burst_Objectives \t Best_Objective")

for burst in range(num_bursts):
    
    MyChain = MarkovChain(
        proposal = proposal1, 
        constraints = [compactness_bound, pop_constraint],
        accept = always_accept, 
        initial_state = incumbent_plan, 
        total_steps = burst_length
    )
    best_maj_min = 0
    all_maj_min = list()
    for plan in MyChain:
        this_maj_min = sum( 1 for j in range(k) if plan['MVAP'][j] >= 0.5 * plan['VAP'][j] )
        all_maj_min.append(this_maj_min)
        if this_maj_min >= best_maj_min:
            incumbent_plan = plan
            best_maj_min = this_maj_min
    
    print(burst,":",all_maj_min,"->",best_maj_min)

Burst 	 Burst_Objectives 	 Best_Objective
0 : [2, 2, 2, 3, 3, 3, 3, 3, 3, 3] -> 3
1 : [3, 3, 3, 3, 3, 3, 3, 3, 3, 3] -> 3
2 : [3, 3, 3, 3, 3, 3, 3, 3, 3, 3] -> 3
3 : [3, 3, 3, 3, 3, 3, 3, 3, 3, 3] -> 3
4 : [3, 3, 3, 3, 3, 3, 3, 3, 3, 3] -> 3
5 : [3, 3, 3, 3, 3, 3, 3, 3, 3, 3] -> 3
6 : [3, 3, 3, 3, 3, 3, 3, 3, 3, 3] -> 3
7 : [3, 3, 3, 3, 3, 3, 3, 3, 3, 3] -> 3
8 : [3, 3, 3, 3, 3, 3, 3, 3, 3, 3] -> 3
9 : [3, 3, 3, 3, 3, 3, 3, 3, 3, 3] -> 3
10 : [3, 3, 3, 3, 3, 3, 3, 3, 3, 3] -> 3
11 : [3, 3, 2, 2, 2, 2, 2, 2, 2, 2] -> 3
12 : [3, 3, 3, 3, 3, 3, 3, 3, 3, 3] -> 3
13 : [3, 3, 3, 3, 3, 3, 3, 3, 3, 3] -> 3
14 : [3, 3, 3, 3, 3, 3, 3, 3, 3, 3] -> 3
15 : [3, 3, 3, 3, 3, 3, 3, 3, 3, 3] -> 3
16 : [3, 3, 3, 3, 3, 3, 3, 3, 3, 3] -> 3
17 : [3, 3, 3, 3, 3, 3, 3, 3, 3, 3] -> 3
18 : [3, 3, 3, 3, 3, 3, 3, 3, 3, 3] -> 3
19 : [3, 3, 3, 3, 3, 3, 3, 3, 3, 3] -> 3
20 : [3, 3, 3, 3, 3, 3, 3, 3, 3, 3] -> 3
21 : [3, 3, 3, 3, 3, 3, 3, 3, 3, 3] -> 3
22 : [3, 3, 3, 3, 3, 3, 3, 3, 3, 3] -> 3
23 : [3, 3, 3, 3, 3, 3, 3

197 : [7, 7, 7, 7, 7, 7, 7, 7, 7, 7] -> 7
198 : [7, 6, 6, 6, 6, 6, 6, 6, 5, 5] -> 7
199 : [7, 7, 7, 7, 7, 7, 7, 7, 7, 7] -> 7
200 : [7, 7, 7, 7, 7, 7, 7, 7, 7, 7] -> 7
201 : [7, 7, 7, 7, 7, 7, 7, 7, 7, 7] -> 7
202 : [7, 7, 7, 7, 7, 7, 7, 7, 7, 7] -> 7
203 : [7, 7, 7, 7, 7, 6, 6, 6, 6, 6] -> 7
204 : [7, 6, 6, 6, 6, 5, 5, 5, 5, 5] -> 7
205 : [7, 7, 7, 7, 7, 7, 7, 6, 6, 6] -> 7
206 : [7, 7, 7, 7, 7, 7, 7, 7, 7, 7] -> 7
207 : [7, 7, 7, 6, 6, 6, 6, 6, 6, 6] -> 7
208 : [7, 7, 6, 6, 6, 6, 6, 5, 5, 5] -> 7
209 : [7, 7, 7, 7, 7, 7, 7, 7, 7, 7] -> 7
210 : [7, 7, 7, 7, 7, 7, 7, 7, 7, 7] -> 7
211 : [7, 6, 6, 6, 6, 6, 6, 6, 6, 6] -> 7
212 : [7, 7, 7, 7, 7, 7, 7, 7, 7, 7] -> 7
213 : [7, 7, 7, 7, 7, 7, 7, 7, 7, 7] -> 7
214 : [7, 7, 7, 7, 7, 7, 7, 7, 7, 6] -> 7
215 : [7, 7, 7, 7, 7, 7, 6, 6, 6, 6] -> 7
216 : [7, 7, 6, 6, 6, 6, 6, 6, 7, 7] -> 7
217 : [7, 7, 7, 7, 7, 7, 7, 7, 7, 7] -> 7
218 : [7, 7, 7, 7, 7, 7, 7, 7, 7, 7] -> 7
219 : [7, 7, 7, 7, 7, 7, 7, 7, 7, 7] -> 7
220 : [7, 7, 7, 7, 7, 7, 6, 6, 6, 

393 : [8, 8, 8, 8, 8, 8, 7, 7, 7, 7] -> 8
394 : [8, 8, 8, 8, 8, 8, 7, 7, 7, 7] -> 8
395 : [8, 8, 8, 8, 8, 8, 8, 8, 8, 8] -> 8
396 : [8, 8, 8, 8, 8, 8, 8, 8, 8, 7] -> 8
397 : [8, 8, 8, 8, 8, 8, 8, 8, 8, 8] -> 8
398 : [8, 8, 8, 8, 8, 8, 8, 8, 8, 8] -> 8
399 : [8, 8, 8, 7, 7, 7, 7, 6, 6, 6] -> 8
400 : [8, 8, 8, 8, 8, 8, 8, 8, 8, 8] -> 8
401 : [8, 8, 8, 8, 8, 8, 7, 7, 7, 7] -> 8
402 : [8, 8, 8, 7, 6, 6, 6, 6, 6, 6] -> 8
403 : [8, 8, 8, 8, 8, 8, 8, 8, 8, 8] -> 8
404 : [8, 8, 7, 7, 7, 7, 7, 7, 7, 7] -> 8
405 : [8, 8, 8, 8, 8, 7, 7, 6, 6, 6] -> 8
406 : [8, 8, 8, 8, 8, 8, 8, 8, 7, 6] -> 8
407 : [8, 8, 8, 8, 8, 8, 8, 7, 7, 7] -> 8
408 : [8, 8, 8, 8, 8, 7, 7, 7, 6, 6] -> 8
409 : [8, 8, 8, 8, 8, 8, 8, 7, 7, 7] -> 8
410 : [8, 8, 8, 8, 8, 8, 8, 8, 8, 8] -> 8
411 : [8, 8, 7, 7, 7, 7, 7, 7, 7, 7] -> 8
412 : [8, 8, 8, 8, 8, 8, 8, 8, 8, 8] -> 8
413 : [8, 8, 8, 8, 8, 8, 8, 8, 8, 8] -> 8
414 : [8, 8, 8, 8, 8, 8, 8, 8, 8, 8] -> 8
415 : [8, 8, 7, 7, 7, 7, 7, 7, 7, 7] -> 8
416 : [8, 8, 8, 8, 8, 8, 8, 8, 8, 

589 : [8, 8, 8, 8, 7, 7, 7, 7, 7, 7] -> 8
590 : [8, 8, 8, 8, 8, 8, 8, 8, 8, 8] -> 8
591 : [8, 8, 8, 8, 8, 8, 8, 8, 8, 8] -> 8
592 : [8, 8, 8, 8, 8, 8, 8, 7, 7, 7] -> 8
593 : [8, 8, 8, 8, 7, 8, 8, 8, 8, 8] -> 8
594 : [8, 8, 8, 8, 7, 7, 7, 7, 7, 7] -> 8
595 : [8, 7, 7, 7, 7, 7, 7, 7, 7, 7] -> 8
596 : [8, 8, 8, 8, 8, 8, 8, 7, 6, 6] -> 8
597 : [8, 8, 8, 8, 8, 8, 8, 8, 8, 8] -> 8
598 : [8, 8, 8, 8, 8, 8, 8, 8, 8, 8] -> 8
599 : [8, 8, 7, 7, 7, 7, 6, 6, 6, 6] -> 8
600 : [8, 8, 8, 7, 7, 6, 6, 6, 6, 6] -> 8
601 : [8, 8, 7, 7, 7, 7, 7, 6, 6, 6] -> 8
602 : [8, 8, 8, 8, 8, 7, 7, 7, 7, 7] -> 8
603 : [8, 8, 8, 8, 8, 8, 8, 8, 8, 8] -> 8
604 : [8, 8, 8, 8, 8, 8, 8, 8, 8, 7] -> 8
605 : [8, 8, 8, 8, 8, 8, 8, 8, 8, 8] -> 8
606 : [8, 7, 7, 6, 6, 6, 6, 6, 6, 6] -> 8
607 : [8, 8, 8, 8, 8, 8, 8, 8, 8, 8] -> 8
608 : [8, 8, 8, 8, 8, 8, 8, 8, 8, 7] -> 8
609 : [8, 7, 7, 7, 7, 7, 7, 6, 6, 6] -> 8
610 : [8, 8, 8, 8, 8, 8, 8, 8, 8, 7] -> 8
611 : [8, 8, 8, 8, 8, 8, 8, 8, 8, 8] -> 8
612 : [8, 8, 8, 8, 8, 8, 8, 7, 7, 

785 : [8, 8, 8, 8, 8, 8, 8, 7, 7, 7] -> 8
786 : [8, 8, 8, 8, 8, 8, 8, 8, 7, 7] -> 8
787 : [8, 8, 8, 8, 8, 8, 8, 8, 8, 8] -> 8
788 : [8, 8, 8, 8, 7, 7, 7, 7, 7, 6] -> 8
789 : [8, 8, 8, 8, 8, 8, 8, 8, 8, 8] -> 8
790 : [8, 8, 8, 8, 7, 7, 7, 7, 7, 7] -> 8
791 : [8, 8, 8, 8, 8, 8, 8, 8, 8, 8] -> 8
792 : [8, 8, 8, 8, 8, 8, 7, 7, 7, 7] -> 8
793 : [8, 8, 8, 8, 7, 7, 7, 7, 7, 7] -> 8
794 : [8, 8, 8, 8, 8, 8, 8, 8, 8, 8] -> 8
795 : [8, 8, 8, 8, 8, 8, 8, 8, 8, 7] -> 8
796 : [8, 8, 8, 8, 8, 8, 8, 8, 8, 8] -> 8
797 : [8, 8, 8, 8, 8, 8, 8, 8, 8, 8] -> 8
798 : [8, 8, 8, 8, 8, 8, 7, 7, 7, 7] -> 8
799 : [8, 8, 8, 8, 8, 8, 8, 8, 8, 8] -> 8
800 : [8, 8, 8, 8, 8, 8, 7, 7, 7, 7] -> 8
801 : [8, 8, 8, 8, 8, 8, 8, 8, 8, 8] -> 8
802 : [8, 8, 8, 8, 8, 7, 7, 7, 7, 7] -> 8
803 : [8, 8, 8, 8, 8, 7, 7, 7, 7, 7] -> 8
804 : [8, 8, 8, 8, 8, 8, 8, 8, 8, 8] -> 8
805 : [8, 8, 8, 8, 8, 8, 8, 7, 7, 7] -> 8
806 : [8, 8, 7, 7, 7, 7, 7, 7, 7, 6] -> 8
807 : [8, 8, 8, 8, 8, 8, 8, 8, 8, 8] -> 8
808 : [8, 8, 8, 8, 8, 8, 8, 8, 8, 

981 : [8, 7, 6, 6, 6, 6, 6, 6, 6, 6] -> 8
982 : [8, 7, 8, 8, 8, 8, 8, 8, 7, 7] -> 8
983 : [8, 8, 8, 8, 8, 8, 7, 6, 6, 6] -> 8
984 : [8, 8, 8, 8, 8, 8, 8, 8, 8, 8] -> 8
985 : [8, 7, 7, 7, 7, 7, 6, 6, 6, 6] -> 8
986 : [8, 8, 8, 8, 8, 8, 8, 8, 8, 8] -> 8
987 : [8, 8, 8, 7, 7, 7, 7, 7, 7, 7] -> 8
988 : [8, 8, 8, 8, 8, 8, 8, 8, 7, 7] -> 8
989 : [8, 8, 8, 7, 7, 7, 7, 7, 7, 7] -> 8
990 : [8, 7, 7, 7, 7, 7, 7, 7, 7, 7] -> 8
991 : [8, 7, 7, 7, 7, 7, 7, 7, 7, 7] -> 8
992 : [8, 8, 8, 8, 8, 8, 8, 8, 8, 8] -> 8
993 : [8, 8, 8, 8, 8, 8, 8, 8, 8, 8] -> 8
994 : [8, 7, 6, 7, 7, 7, 7, 7, 7, 7] -> 8
995 : [8, 8, 8, 8, 8, 8, 8, 8, 8, 8] -> 8
996 : [8, 8, 8, 7, 6, 6, 6, 6, 6, 6] -> 8
997 : [8, 7, 7, 8, 8, 8, 8, 8, 8, 8] -> 8
998 : [8, 8, 8, 8, 8, 8, 8, 8, 8, 8] -> 8
999 : [8, 8, 7, 6, 6, 6, 6, 6, 6, 6] -> 8


In [11]:
print_stats(incumbent_plan)

# mvap vap mvap%
0 17472 94840 18.42 %
1 59853 89681 66.74 %
2 25048 91017 27.52 %
3 15361 84686 18.14 %
4 21791 91747 23.75 %
5 20187 88677 22.76 %
6 48845 94437 51.72 %
7 25337 94754 26.74 %
8 8006 88198 9.08 %
9 45678 89986 50.76 %
10 47643 90669 52.55 %
11 21891 89546 24.45 %
12 17072 90653 18.83 %
13 11661 95334 12.23 %
14 13865 94572 14.66 %
15 28540 92220 30.95 %
16 5576 85782 6.5 %
17 39202 92075 42.58 %
18 28336 101576 27.9 %
19 17225 91254 18.88 %
20 20817 87543 23.78 %
21 40545 94448 42.93 %
22 45337 89804 50.48 %
23 27613 92822 29.75 %
24 28469 88487 32.17 %
25 13973 92742 15.07 %
26 36862 86970 42.38 %
27 46217 89419 51.69 %
28 16258 93488 17.39 %
29 29209 88574 32.98 %
30 32178 95677 33.63 %
31 46858 91938 50.97 %
32 30378 89167 34.07 %
33 35312 101807 34.69 %
34 63523 99616 63.77 %
35 20545 87989 23.35 %
36 32646 91499 35.68 %
37 19498 89607 21.76 %
38 10942 87247 12.54 %
Number of majority-minority districts: 8


In [10]:
districts = [ list(incumbent_plan.parts[j]) for j in range(k) ]
print("districts =",districts)

districts = [[0, 2, 67, 4, 68, 70, 7, 72, 6, 8, 11, 12, 15, 19, 20, 25, 26, 28, 30, 31, 115, 116, 54, 119, 120, 118, 506, 123, 511], [768, 769, 320, 323, 772, 581, 773, 774, 776, 840, 587, 780, 1227, 973, 591, 78, 975, 1276, 832, 284, 287, 231, 426, 239, 240, 1010, 315, 956], [640, 642, 646, 1290, 1291, 1292, 1293, 531, 279, 280, 931, 932, 39, 682, 683, 1331, 52, 1084, 705, 450, 707, 451, 710, 583, 331, 460, 333, 334, 335, 336, 337, 338, 339, 340, 466, 637, 639], [255, 261, 779, 289, 291, 292, 293, 296, 297, 298, 299, 300, 232, 302, 303, 304, 308, 309, 310, 248, 253, 254, 831], [1026, 1028, 1029, 390, 1033, 1034, 1035, 1036, 1037, 1038, 167, 168, 810, 433, 829, 830, 971, 205, 206, 726, 727, 728, 729, 470, 731, 730, 734, 735, 736, 737, 738, 739, 749], [128, 385, 384, 394, 395, 533, 534, 1052, 158, 43, 1072, 1074, 53, 208, 209, 346, 612, 872, 363, 632, 877, 887, 879, 880, 883, 886, 631, 888, 890, 891, 125, 126], [256, 833, 258, 771, 260, 322, 324, 453, 839, 1226, 782, 720, 721, 722, 767,